# `DataFrame`

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = (
    SparkConf()
        .setAppName("Spark SQL Course")
)

sc = SparkContext(conf=conf)  # no need for Spark 3...

spark = (
    SparkSession
        .builder
        .appName("Spark SQL Course")
        .getOrCreate()
)

In [3]:
from pyspark.sql import Row

row1 = Row(name="John", age=21)
row2 = Row(name="James", age=32)
row3 = Row(name="Jane", age=18)
row1['name']

In [4]:
df = spark.createDataFrame([row1, row2, row3])

In [5]:
df.printSchema()

How does `printSchema` compare with Pandas `info()`?

In [6]:
df.show()

How does `show` compare with Pandas `head()`

In [7]:
print(df.rdd.toDebugString().decode("utf-8"))

In [8]:
df.rdd.getNumPartitions()

## Creating dataframes

In [9]:
rows = [
    Row(name="John", age=21, gender="male"),
    Row(name="James", age=25, gender="female"),
    Row(name="Albert", age=46, gender="male")
]

df = spark.createDataFrame(rows)

In [10]:
df.show()

In [11]:
help(Row)

In [12]:
column_names = ["name", "age", "gender"]
rows = [
    ["John", 21, "male"],
    ["James", 25, "female"],
    ["Albert", 46, "male"]
]

df = spark.createDataFrame(
    rows, 
    column_names
)

df.show()

In [13]:
df.printSchema()

In [14]:
# sc = SparkContext(conf=conf)  # no need for Spark 3...

column_names = ["name", "age", "gender"]
rdd = sc.parallelize([
    ("John", 21, "male"),
    ("James", 25, "female"),
    ("Albert", 46, "male")
])
df = spark.createDataFrame(rdd, column_names)
df.show()

## Schema

There is special type schemata. A object of class `StructType` is made
of a list of objects of type `StructField`.

In [15]:
df.schema

How does `df.schema` relate to `df.printSchema()`? Where would you use
the outputs of `df.schema` and `df.printSchema()`.

In [16]:
type(df.schema)

A object of type `StructField` has a *name* like `gender`, a PySpark
*type* like `StringType()`, an d a boolean parameter.

What does the boolean parameter stand for?

In [17]:
from pyspark.sql.types import *

schema = StructType(
    [
        StructField("name", StringType(), False),
        StructField("age", IntegerType(), True),
        StructField("gender", StringType(), True)
    ]
)

rows = [("Jane", 21, "female")]
df = spark.createDataFrame(rows, schema)
df.printSchema()
df.show()

# Queries (single table $σ$, $π$)

PySpark offers two ways to query a datafrane:

-   An ad hoc API with methods for the DataFrame class.
-   The possibility to post SQL queries (provided a temporary view has
    been created).

In [18]:
column_names = ["name", "age", "gender"]
rows = [
    ["John", 21, "male"],
    ["Jane", 25, "female"]
]
# 
df = spark.createDataFrame(rows, column_names)

# Create a temporary view from the DataFrame
df.createOrReplaceTempView("new_view")

# Apply the query
query = """
    SELECT 
        name, age 
    FROM 
        new_view 
    WHERE 
        gender='male'
"""

men_df = spark.sql(query)
men_df.show()

> **New! (with Spark 4)**
>
> We can now write SQL queries the way we have been writing in the
> `tidyverse` (`R`) using the SQL pipe `|>`.

In [19]:
new_age_query = """
    FROM new_view
    |> WHERE gender = 'male'
    |> SELECT name, age
"""

to be compared with

``` r
new_df |> 
    dplyr::filter(gender=='male') |>
    dplyr::select(name, age)
```

In [20]:
men_df = (
    spark
        .sql(new_age_query)
        .show()
)

> **Have a look at**
>
> <https://www.databricks.com/blog/sql-gets-easier-announcing-new-pipe-syntax>

## `SELECT` (projection $π$)

In [21]:
df.createOrReplaceTempView("table")    

query = """
    SELECT 
        name, age 
    FROM 
        table
"""

spark.sql(query).show()

Using the API:

In [22]:
(
    df
        .select("name", "age")
        .show()
)

`π(df, "name", "age")`

## `WHERE` (filter, selection, $σ$)

In [23]:
df.createOrReplaceTempView("table")

query = """
    SELECT 
        * 
    FROM 
        table
    WHERE 
        age > 21
"""

query = """
    FROM table 
    |> WHERE age > 21 
    |> SELECT *   
"""

spark.sql(query).show()

Note that you can get rid of `|> SELECT *`

Using the API

In [24]:
( 
    df
        .where("age > 21")
        .show()
)

This implements `σ(df, "age > 21")`

The `where()` method takes different types of inputs as argument:
strings that can be interpreted as SQL conditions, but also boolean
masks.

In [25]:
# Alternatively:
( 
    df
      .where(df['age'] > 21)
      .show()
)

or

In [26]:
( 
    df
      .where(df.age > 21)
      .show()
)

Where (and how) is the boolean mask built?

Method chaining allows to construct complex queries

In [27]:
( 
    df
      .where("age > 21")
      .select(["name", "age"])
      .show()
)

This implements

        σ(df, "age > 21") |>
        π(["name", "age"])

## `LIMIT`

In [28]:
df.createOrReplaceTempView("table")

query = """
    SELECT 
        * 
    FROM 
        table 
    LIMIT 1
"""

query = """
    FROM table
    |> LIMIT 1
"""

spark.sql(query).show()

In [29]:
df.limit(1).show()

In [30]:
df.select("*").limit(1).show()

## `ORDER BY`

In [31]:
df.createOrReplaceTempView("table")

query = """
    SELECT 
        * 
    FROM 
        table
    ORDER BY 
        name ASC
"""

query = """
    FROM table
    |> ORDER BY name ASC
"""

spark.sql(query).show()

With the API

In [32]:
df.orderBy(df.name.asc()).show()

## `ALIAS` (rename)

In [33]:
df.createOrReplaceTempView("table")

query = """
    SELECT 
        name, age, gender  AS sex 
    FROM 
        table
"""

query = """
    FROM table
    |> SELECT name, age, gender  AS sex
"""

spark.sql(query).show()

In [34]:
type(df.age)

In [35]:
(
    df.select(
        df.name, 
        df.age, 
        df.gender.alias('sex'))
      .show()
)

## `CAST`

In [36]:
df.createOrReplaceTempView("table")

query = """
    SELECT 
        name, 
        cast(age AS float) AS age_f 
    FROM 
        table
"""

query = """
    FROM table |>  
    SELECT 
        name, 
        cast(age AS float) AS age_f 
"""

spark.sql(query).show()

In [37]:
(
    df
        .select(
            df.name, 
            df.age
                .cast("float")
                .alias("age_f"))
        .show()
)

In [38]:
new_age_col = df.age.cast("float").alias("age_f")
type(new_age_col), type(df.age)

In [39]:
df.select(df.name, new_age_col).show()

## Adding new columns

In [40]:
df.createOrReplaceTempView("table")

query = """
    FROM table |>
    SELECT 
        *, 
        12*age AS age_months 
"""

spark.sql(query).show()

In [41]:
( 
    df
        .withColumn("age_months", df.age * 12)
        .show()
)

In [42]:
(
    df
        .select("*", 
                (df.age * 12).alias("age_months"))
        .show()
)

In [43]:
import datetime

hui = datetime.date.today()

str(hui)

In [44]:
( 
    df
     .withColumn("yob", datetime.date.today().year - df.age)
     .show()
)

# Column functions

## Numeric functions examples

In [45]:
from pyspark.sql import functions as fn

columns = ["brand", "cost"]

df = spark.createDataFrame(
    [("garnier", 3.49),
     ("elseve", 2.71)], 
    columns
)

round_cost = fn.round(df.cost, 1)
floor_cost = fn.floor(df.cost)
ceil_cost = fn.ceil(df.cost)

(
    df
    .withColumn('round', round_cost)
    .withColumn('floor', floor_cost)
    .withColumn('ceil', ceil_cost)
    .show()
)

## String functions examples

In [46]:
from pyspark.sql import functions as fn

columns = ["first_name", "last_name"]

df = spark.createDataFrame([
    ("John", "Doe"),
    ("Mary", "Jane")
], columns)

last_name_initial = fn.substring(df.last_name, 0, 1)
# last_name_initial_dotted = fn.concat(last_name_initial, ".")

name = fn.concat_ws(" ", df.first_name, last_name_initial)

df.withColumn("name", name).show()

In [47]:
( 
    df.selectExpr("*", "substring(last_name, 0, 1) as lni")
      .selectExpr("first_name", "last_name", "concat(first_name, ' ', lni, '.') as nname")
      .show()
)

As an SQL query

In [48]:
df.createOrReplaceTempView("table")

query = """
    FROM table |>
    SELECT *, substring(last_name, 0, 1) AS lni |>
    SELECT first_name, last_name, concat(first_name, ' ', lni, '.') AS nname
"""
spark.sql(query).show()

Spark SQL offers a large subsets of [SQL
functions](https://www.postgresql.org/docs/current/functions.html)

## Date functions examples

In [49]:
from datetime import date
from pyspark.sql import functions as fn

df = spark.createDataFrame([
    (date(2015, 1, 1), date(2015, 1, 15)),
    (date(2015, 2, 21), date(2015, 3, 8)),
], ["start_date", "end_date"])

days_between = fn.datediff(df.end_date, df.start_date)
start_month = fn.month(df.start_date)

(
    df
        .withColumn('days_between', days_between)
        .withColumn('start_month', start_month)
        .show()
)

Note that `days_between` is an instance of `Column`, the Spark type for
columns in Spark dataframes.

In [50]:
# %%
type(days_between)

Recall the datetime calculus available in Database systems and in many
programming framework.

In [51]:
str(date(2015, 1, 1) - date(2015, 1, 15))

In [52]:
from datetime import timedelta

date(2023, 2 , 14) + timedelta(days=3)

## Conditional transformations

In [53]:
df = spark.createDataFrame([
    ("John", 21, "male"),
    ("Jane", 25, "female"),
    ("Albert", 46, "male"),
    ("Brad", 49, "super-hero")
], ["name", "age", "gender"])

In [54]:
supervisor = ( 
    fn.when(df.gender == 'male', 'Mr. Smith')
      .when(df.gender == 'female', 'Miss Jones')
      .otherwise('NA')
)

type(supervisor), type(fn.when)

In [55]:
df.withColumn("supervisor", supervisor).show()

## User-defined functions

In [56]:
from pyspark.sql import functions as fn
from pyspark.sql.types import StringType

df = spark.createDataFrame([(1, 3), (4, 2)], ["first", "second"])

def my_func(col_1, col_2):
    if (col_1 > col_2):
        return "{} is bigger than {}".format(col_1, col_2)
    else:
        return "{} is bigger than {}".format(col_2, col_1)

# registration
my_udf = fn.udf(my_func, StringType())

# fn.udf is a decorator
# it can be used in a a more explicit way

@fn.udf(returnType=StringType())
def the_same_func(col_1, col_2):
    if (col_1 > col_2):
        return "{} is bigger than {}".format(col_1, col_2)
    else:
        return "{} is bigger than {}".format(col_2, col_1)

# at work        
(
    df
        .withColumn("udf", my_udf(df['first'], df['second']))
        .withColumn("udf_too", the_same_func(df['first'], df['second']))
        .show()
)

> **Using UDF in SQL queries**
>
> The user-defined-function (UDF) can also be used on SQL queries
> provided the decorated function is registered.
>
> ``` python
> df.createOrReplaceTempView("table")
>
> spark.udf.register("the_same_func", the_same_func)  # the_same_func from @udf above
> spark.sql("SELECT *, the_same_func(first, second) AS udf FROM table").show()
> ```
>
> Beware. `spark.udf.register` is not a decorator.

> **More on UDF in Spark 4**
>
> See [User-Defined Table Functions (UDTFs) in
> Python](https://docs.databricks.com/gcp/en/udf/python-udtf)

# Joins ($⋈$)

## Using the `spark.sql` API

In [58]:
from datetime import date

products = spark.createDataFrame(
    [
        ('1', 'mouse', 'microsoft', 39.99),
        ('2', 'keyboard', 'logitech', 59.99),
    ], 
    ['prod_id', 'prod_cat', 'prod_brand', 'prod_value']
)

purchases = spark.createDataFrame([
    (date(2017, 11, 1), 2, '1'),
    (date(2017, 11, 2), 1, '1'),
    (date(2017, 11, 5), 1, '2'),
], ['date', 'quantity', 'prod_id'])

# The default join type is the "INNER" join
purchases.join(products, 'prod_id').show()

Just as in RDBMs, we can ask for explanations:

In [59]:
purchases.join(products, 'prod_id').explain()

Have a look at the PostgreSQL documentation. What are the different join
methods? When is one favored?

## Using a `SQL` query

In [60]:
products.createOrReplaceTempView("products")
purchases.createOrReplaceTempView("purchases")

query = """
    SELECT * 
    FROM purchases AS prc INNER JOIN 
        products AS prd 
    ON prc.prod_id = prd.prod_id
"""

query = """
    FROM purchases |>
    INNER JOIN products USING(prod_id)    
"""

spark.sql(query).show()

In [61]:
spark.sql(query).explain()

In [62]:
new_purchases = spark.createDataFrame([
    (date(2017, 11, 1), 2, '1'),
    (date(2017, 11, 2), 1, '3'),
], ['date', 'quantity', 'prod_id_x'])

# The default join type is the "INNER" join
join_rule = new_purchases.prod_id_x == products.prod_id

print(type(join_rule))

new_purchases.join(products, join_rule, 'left').show()

What is the type of `join_rule.info`?

In [63]:
join_rule.info

In [64]:
new_purchases = spark.createDataFrame([
    (date(2017, 11, 1), 2, '1'),
    (date(2017, 11, 2), 1, '3'),
], ['date', 'quantity', 'prod_id_x'])

# The default join type is the "INNER" join
join_rule = new_purchases.prod_id_x == products.prod_id

new_purchases.join(products, join_rule, 'left').show()

> **Different kinds of joins**
>
> Just as in RDBMS, there are different kinds of joins.
>
> They differ in the way tuples from left and the right tables are
> matched (is it a natural join, an equi-join, a $\theta$ join?). They
> also differ in the way they handle non-matching tuples (inner or outer
> joins).
>
> The `join` method has three parameters:
>
> -   `other`: the right table
> -   `on`: the join rule that defines how tuples are matched.
> -   `how`: defines the way non-matching tuples are handled.

## Various types of joins

In [65]:
left = spark.createDataFrame([
    (1, "A1"), (2, "A2"), (3, "A3"), (4, "A4")], 
    ["id", "value"])

right = spark.createDataFrame([
    (3, "A3"), (4, "A4"), (4, "A4_1"), (5, "A5"), (6, "A6")], 
    ["id", "value"])

join_types = [
    "inner", "outer", "left", "right",
    "leftsemi", "leftanti"
]

In [66]:
for join_type in join_types:
    print(join_type)
    left.join(right, on="id", how=join_type)\
        .orderBy("id")\
        .show()

# Agregations (summarize)

## Examples using the API

In [67]:
from pyspark.sql import functions as fn

products = spark.createDataFrame([
    ('1', 'mouse', 'microsoft', 39.99),
    ('2', 'mouse', 'microsoft', 59.99),
    ('3', 'keyboard', 'microsoft', 59.99),
    ('4', 'keyboard', 'logitech', 59.99),
    ('5', 'mouse', 'logitech', 29.99),
], ['prod_id', 'prod_cat', 'prod_brand', 'prod_value'])

( 
    products
        .groupBy('prod_cat')
        .avg('prod_value')
        .show()
)

What is the type of `products         .groupBy('prod_cat')`?

In [68]:
(
    products
        .groupBy('prod_cat')
        .agg(fn.avg('prod_value'))
        .show()
)

In [69]:
(
    products
        .groupBy('prod_cat')
        .agg(
            fn.mean('prod_value'), 
            fn.stddev('prod_value')
        )
        .show()
)

In [70]:
from pyspark.sql import functions as fn

(
    products
        .groupBy('prod_brand', 'prod_cat')\
        .agg(
            fn.avg('prod_value')
        )
        .show()
)

In [71]:
from pyspark.sql import functions as fn

(
    products
        .groupBy('prod_brand')
        .agg(
            fn.round(
                fn.avg('prod_value'), 1)
                .alias('average'),
            fn.ceil(
                fn.sum('prod_value'))
                .alias('sum'),
            fn.min('prod_value')
                .alias('min')
        )
        .show()
)

## Example using a query

In [72]:
products.createOrReplaceTempView("products")

In [73]:
query = """
SELECT
    prod_brand,
    round(avg(prod_value), 1) AS average,
    min(prod_value) AS min
FROM 
    products
GROUP BY 
    prod_brand
"""

query = """
    FROM products 
    |> AGGREGATE 
        round(avg(prod_value), 1) AS average,
        min(prod_value) AS min GROUP BY prod_brand 
"""

spark.sql(query).show()

In [74]:
spark.sql(query).explain()

# Window functions

## Numerical window functions

In [75]:
from pyspark.sql import Window
from pyspark.sql import functions as fn

# First, we create the Window definition
window = Window.partitionBy('prod_brand')

print(type(window))

Then, we can use `over` to aggregate on this window

In [76]:
avg = fn.avg('prod_value').over(window)

# Finally, we can it as a classical column
(
    products
        .withColumn('avg_brand_value', fn.round(avg, 2))
        .show()
)

With SQL queries, using windows ?

In [77]:
query = """
    SELECT 
        *, 
        ROUND(AVG(prod_value) OVER w1, 2)  AS avg_brand_value,
        ROUND(AVG(prod_value) OVER w2, 1)  AS avg_prod_value
    FROM 
        products
    WINDOW 
        w1 AS (PARTITION BY prod_brand),
        w2 AS (PARTITION BY prod_cat)
"""


query2 = """
    FROM products |>
    SELECT
        *,  
        ROUND(AVG(prod_value) OVER w1, 2)  AS avg_brand_value,
        ROUND(AVG(prod_value) OVER w2, 1)  AS avg_prod_value
        WINDOW 
            w1 AS (PARTITION BY prod_brand),
            w2 AS (PARTITION BY prod_cat)
"""

spark.sql(query2).show()

In [78]:
window2 = Window.partitionBy('prod_cat')

avg2 = fn.avg('prod_value').over(window2)

# Finally, we can do it as a classical column
( 
    products
        .withColumn('avg_brand_value', fn.round(avg, 2))
        .withColumn('avg_prod_value', fn.round(avg2, 1))
        .show()
)

Now we can compare the physical plans associated with the two jobs.

In [79]:
( 
    products
        .withColumn('avg_brand_value', fn.round(avg, 2))
        .withColumn('avg_prod_value', fn.round(avg2, 1))
        .explain()
)

In [80]:
spark.sql(query).explain()

# Windows can be defined on multiple columns

In [81]:
from pyspark.sql import Window
from pyspark.sql import functions as fn

window = Window.partitionBy('prod_brand', 'prod_cat')

avg = fn.avg('prod_value').over(window)


(
    products    
        .withColumn('avg_value', fn.round(avg, 2))
        .show()
)

## Lag and Lead

In [82]:
purchases = spark.createDataFrame(
    [
        (date(2017, 11, 1), 'mouse'),
        (date(2017, 11, 2), 'mouse'),
        (date(2017, 11, 4), 'keyboard'),
        (date(2017, 11, 6), 'keyboard'),
        (date(2017, 11, 9), 'keyboard'),
        (date(2017, 11, 12), 'mouse'),
        (date(2017, 11, 18), 'keyboard')
    ], 
    ['date', 'prod_cat']
)

purchases.show()

window = Window.partitionBy('prod_cat').orderBy('date')

prev_purch = fn.lag('date', 1).over(window)
next_purch = fn.lead('date', 1).over(window)

purchases\
    .withColumn('prev', prev_purch)\
    .withColumn('next', next_purch)\
    .orderBy('prod_cat', 'date')\
    .show()

## Rank, DenseRank and RowNumber

In [83]:
contestants = spark.createDataFrame(
    [   
        ('veterans', 'John', 3000),
        ('veterans', 'Bob', 3200),
        ('veterans', 'Mary', 4000),
        ('young', 'Jane', 4000),
        ('young', 'April', 3100),
        ('young', 'Alice', 3700),
        ('young', 'Micheal', 4000),
    ], 
    ['category', 'name', 'points']
)

contestants.show()

In [84]:
window = (
    Window
        .partitionBy('category')
        .orderBy(contestants.points.desc())
)

rank = fn.rank().over(window)
dense_rank = fn.dense_rank().over(window)
row_number = fn.row_number().over(window)

(
contestants
    .withColumn('rank', rank)
    .withColumn('dense_rank', dense_rank)
    .withColumn('row_number', row_number)
    .orderBy('category', fn.col('points').desc())
    .show()
)